# تحليل البيانات أحادي المتغير (Univariate Analysis)

**الهدف**: فهم توزيع المتغيرات الفردية (المبيعات، الأسعار، الفئات) باستخدام PySpark للتعامل مع البيانات الضخمة (58 مليون صف).

> **ملاحظة هامة**: تم تشغيل هذا الملف باستخدام بيئة Conda (pyspark_env).

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# إعداد الجلسة (Spark Session)
spark = SparkSession.builder \
    .appName("M5 Univariate Analysis") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.python.worker.faulthandler.enabled", "true") \
    .getOrCreate()

print("تم إنشاء جلسة Spark بنجاح")

In [ ]:
# تحميل البيانات
df = spark.read.parquet("train.parquet")
df.printSchema()

## التحليل أ: توزيع المبيعات العالمية (Global Sales Distribution)

**الاستراتيجية**:
1. **التصفية (Filter)**: استبعاد القيم الصفرية `sales == 0` (حيث تمثل حوالي 70% من البيانات).
2. **التقسيم (Binning)**: استخدام دالة `.histogram()` لتقسيم البيانات داخل Spark.
3. **التصور (Visualize)**: رسم مخطط شريطي باستخدام مقياس لوغاريتمي (Log Scale).

In [ ]:
# 1. تصفية المبيعات الصفرية (Sampling for robustness)
# Using sampling and Pandas to avoid RDD serialization issues in this env
pdf = df.filter(col('sales') > 0).select('sales').sample(fraction=0.1).limit(100000).toPandas()

# 2. الرسم
plt.figure(figsize=(12, 6))
plt.hist(pdf['sales'], bins=50, log=True, alpha=0.7)
plt.title('توزيع المبيعات العالمية (مقياس لوغاريتم - عينة)')
plt.xlabel('وحدات المبيعات')
plt.ylabel('العدد (Log Scale)')
plt.grid(axis='y', which='both', linestyle='--', alpha=0.5)
plt.show()


## التحليل ب: توزيع الأسعار (Price Distribution)

**الاستراتيجية**:
1. **القيم المميزة**: اختيار القيم الفريدة فقط للثنائي `(item_id, sell_price)` لتقليل حجم البيانات.
2. **التجميع**: تحويل النتيجة الصغيرة إلى Pandas.
3. **التصور**: رسم مخطط كثافة التقدير (KDE).

In [ ]:
# 1. استعلام القيم المميزة
distinct_prices_df = df.select("item_id", "sell_price") \
    .filter(col("sell_price").isNotNull()) \
    .distinct()

# 2. التحويل إلى Pandas
pdf_prices = distinct_prices_df.toPandas()
print(f"عدد نقاط السعر المميزة: {len(pdf_prices)}")

# 3. الرسم (KDE)
plt.figure(figsize=(12, 6))
sns.kdeplot(data=pdf_prices, x='sell_price', fill=True, color='purple')
plt.title("توزيع الأسعار (للمنتجات المميزة)")
plt.xlabel("سعر البيع ($)")
plt.grid(True, alpha=0.3)
plt.show()

## التحليل ج: توازن الفئات (Category Balance)

**الاستراتيجية**:
1. **التجميع (Aggregation)**: التجميع حسب `cat_id` والعد.
2. **التصور**: رسم مخطط شريطي لعدد المنتجات في كل فئة.

In [ ]:
# 1. التجميع والعد
cat_counts = df.groupBy("cat_id").count().orderBy(desc("count"))

# التحويل إلى Pandas
pdf_cat = cat_counts.toPandas()

# 2. الرسم
plt.figure(figsize=(10, 6))
sns.barplot(data=pdf_cat, x='cat_id', y='count', palette='viridis')
plt.title("توازن الفئات (إجمالي الصفوف)")
plt.xlabel("الفئة")
plt.ylabel("العدد")
plt.show()